10.1

In [1]:
from numpy import *

def loadDataSet(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float,curLine) #map all elements to float()
        dataMat.append(fltLine)
    return dataMat

def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))#create centroid mat
    for j in range(n):#create random cluster centers, within bounds of each dimension
        minJ = min(dataSet[:,j]) 
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = mat(minJ + rangeJ * random.rand(k,1))
    return centroids

In [2]:
datMat = mat(loadDataSet('testSet.txt'))

In [3]:
min(datMat[:, 0])

matrix([[-5.379713]])

In [4]:
min(datMat[:, 1])

matrix([[-4.232586]])

In [5]:
max(datMat[:, 1])

matrix([[5.1904]])

In [6]:
max(datMat[:, 0])

matrix([[4.838138]])

In [7]:
randCent(datMat, 2)

matrix([[-2.82670195,  0.5171291 ],
        [ 3.74860199,  1.58010375]])

In [8]:
distEclud(datMat[0], datMat[1])

5.184632816681332

In [9]:
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]#get all the point in this cluster
            centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean 
    return centroids, clusterAssment

In [10]:
datMat = mat(loadDataSet('testSet.txt'))

In [11]:
myCentroids, clustAssing = kMeans(datMat, 4)

[[ 0.02992029  4.79153623]
 [ 4.19293383  1.35876666]
 [-4.00655666  5.13561387]
 [-3.82942199  3.83865895]]
[[ 0.86388509  3.82115945]
 [ 2.75739947 -0.73853488]
 [        nan         nan]
 [-3.18695357 -0.35938491]]
[[ 1.20648696  3.31008096]
 [ 2.723454   -2.26243579]
 [        nan         nan]
 [-3.34884281 -0.77772481]]
[[ 1.21336621  3.14825539]
 [ 2.72102136 -2.61215086]
 [        nan         nan]
 [-3.4041927  -0.8779912 ]]
[[ 1.18693127  3.07207553]
 [ 2.65077367 -2.79019029]
 [        nan         nan]
 [-3.43321524 -1.00773014]]
[[ 0.94953884  3.05400984]
 [ 2.65077367 -2.79019029]
 [        nan         nan]
 [-3.49409433 -1.28852678]]
[[ 0.48673911  3.068488  ]
 [ 2.65077367 -2.79019029]
 [        nan         nan]
 [-3.54251791 -2.066412  ]]
[[ 0.34421986  3.07632743]
 [ 2.65077367 -2.79019029]
 [        nan         nan]
 [-3.4859745  -2.31300105]]
[[ 0.14460654  3.09399208]
 [ 2.72102136 -2.61215086]
 [        nan         nan]
 [-3.50009376 -2.50791867]]
[[ 0.0469085   3.05

/home/zog/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/matrixlib/defmatrix.py:442: RuntimeWarning: Mean of empty slice.
  return N.ndarray.mean(self, axis, dtype, out, keepdims=True)._collapse(axis)
/home/zog/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


[[-0.02298687  2.99472915]
 [ 2.8692781  -2.54779119]
 [        nan         nan]
 [-3.38237045 -2.9473363 ]]


10.3

In [12]:
def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
    centList =[centroid0] #create a list with one centroid
    for j in range(m):#calc initial Error
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]#get the data points currently in cluster i
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])
            print "sseSplit, and notSplit: ",sseSplit,sseNotSplit
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print 'the bestCentToSplit is: ',bestCentToSplit
        print 'the len of bestClustAss is: ', len(bestClustAss)
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids 
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return mat(centList), clusterAssment

In [13]:
datMat3 = mat(loadDataSet('testSet2.txt'))

In [14]:
centList, myNewAssments = biKmeans(datMat3, 3)

[[ 3.21238187 -1.2932741 ]
 [-4.47029841  0.17578531]]
[[ 1.72116203  0.59114988]
 [-2.61472592  2.05464273]]
[[ 1.47897892  0.34916627]
 [-2.79067761  2.63481109]]
[[ 1.23710375  0.17480612]
 [-2.94737575  3.3263781 ]]
sseSplit, and notSplit:  570.7227574246755 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  60
[[ 0.74109018  4.36519284]
 [-0.70395033 -1.63231039]]
[[ 2.95977168  3.26903847]
 [-0.32150057 -2.62473743]]
[[ 2.93386365  3.12782785]
 [-0.45965615 -2.7782156 ]]
sseSplit, and notSplit:  68.68654812621844 38.06295063565756
[[-4.28637538  2.84177688]
 [-3.06866781  4.62314706]]
[[-3.29599467  2.62429878]
 [-2.66214209  3.90080664]]
[[-3.16218625  2.44942162]
 [-2.80416875  3.91101575]]
sseSplit, and notSplit:  27.193667836261877 532.6598067890178
the bestCentToSplit is:  0
the len of bestClustAss is:  40


In [15]:
centList

matrix([[ 2.93386365,  3.12782785],
        [-2.94737575,  3.3263781 ],
        [-0.45965615, -2.7782156 ]])